In [ ]:
# Install required libraries
!pip install requests pandas matplotlib yfinance langchain spacy
!python -m spacy download en_core_web_lg

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import spacy
from spacy.matcher import Matcher
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from typing import Dict, List, Any, Tuple

In [ ]:
class QueryProcessor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_lg")
        self.matcher = Matcher(self.nlp.vocab)
        self.company_to_ticker = self._initialize_company_database()
        self._create_metric_patterns()

    def _initialize_company_database(self) -> Dict[str, str]:
        return {
            "tesla": "TSLA",
            "apple": "AAPL",
            "nvidia": "NVDA",
            "microsoft": "MSFT",
            "amazon": "AMZN",
            "google": "GOOGL",
            "alphabet": "GOOGL",
            "meta": "META",
            "facebook": "META"
        }

    def _create_metric_patterns(self):
        self.matcher.add("REVENUE", [[{"LOWER": "revenue"}]])
        self.matcher.add("EBITDA", [[{"LOWER": "ebitda"}]])
        self.matcher.add("EPS", [[{"LOWER": "eps"}]])
        self.matcher.add("PE_RATIO", [[{"LOWER": "p/e"}, {"LOWER": "ratio"}]])
        self.matcher.add("INTEREST_RATE", [[{"LOWER": "interest"}, {"LOWER": "rate"}]])

    def extract_companies(self, query: str) -> List[Dict[str, str]]:
        doc = self.nlp(query.lower())
        companies = []

        for ent in doc.ents:
            if ent.label_ == "ORG":
                company_name = ent.text.lower()
                if company_name in self.company_to_ticker:
                    companies.append({
                        "name": company_name,
                        "ticker": self.company_to_ticker[company_name]
                    })

        ticker_pattern = r'\b[A-Z]{1,5}\b'
        potential_tickers = re.findall(ticker_pattern, query)
        for ticker in potential_tickers:
            companies.append({"name": ticker, "ticker": ticker})

        return companies

    def extract_metrics(self, query: str) -> List[str]:
        doc = self.nlp(query)
        matches = self.matcher(doc)

        metrics = []
        for match_id, start, end in matches:
            span = doc[start:end]
            metrics.append(span.text)

        return metrics

    def process_query(self, query: str) -> Dict:
        companies = self.extract_companies(query)
        metrics = self.extract_metrics(query)
        
        return {
            "companies": companies,
            "metrics": metrics,
            "query_text": query
        }

query_processor = QueryProcessor()
example_query = "Summarize TSLA's financials over the last 5 years"
processed_query = query_processor.process_query(example_query)
print(processed_query)

In [ ]:
class APIConnector:
    def __init__(self, api_keys: Dict[str, str]):
        self.api_keys = api_keys

    def fetch_alpha_vantage_data(self, symbol: str) -> Dict:
        base_url = "https://www.alphavantage.co/query"
        
        params = {
            "function": "OVERVIEW",
            "symbol": symbol,
            "apikey": self.api_keys["alpha_vantage"]
        }
        
        response = requests.get(base_url, params=params)
        
        return response.json()

    def fetch_yahoo_finance_data(self, symbol: str) -> pd.DataFrame:
        stock = yf.Ticker(symbol)
        
        return stock.history(period="5y")

api_keys = {"alpha_vantage": "<YOUR_ALPHA_VANTAGE_API_KEY>"}
api_connector = APIConnector(api_keys)

# Example usage
alpha_vantage_data = api_connector.fetch_alpha_vantage_data("TSLA")
yahoo_finance_data = api_connector.fetch_yahoo_finance_data("TSLA")

print(alpha_vantage_data)
print(yahoo_finance_data.head())

In [ ]:
class SECFilingAgent:
    def analyze_sec_filings(self, data: Dict):
        return {"insights": f"Analyzed SEC filings for {data.get('Name', 'Unknown')}."}

class MarketResearchAgent:
    def gather_market_context(self, data: pd.DataFrame):
        return {"market_context": f"Market data analyzed. Historical trends observed."}

class TechnicalAnalysisAgent:
    def perform_technical_analysis(self, data: pd.DataFrame):
        return {"technical_analysis": f"Technical indicators calculated."}

sec_agent = SECFilingAgent()
market_agent = MarketResearchAgent()
technical_agent = TechnicalAnalysisAgent()

sec_analysis_result = sec_agent.analyze_sec_filings(alpha_vantage_data)
market_context_result = market_agent.gather_market_context(yahoo_finance_data)
technical_analysis_result = technical_agent.perform_technical_analysis(yahoo_finance_data)

print(sec_analysis_result)
print(market_context_result)
print(technical_analysis_result)

In [ ]:
class ReportGenerator:
    def generate_report(self, sec_analysis: Dict, market_context: Dict, technical_analysis: Dict) -> str:
        report = f"""
Financial Analysis Report:
===========================
SEC Analysis:
{sec_analysis['insights']}

Market Context:
{market_context['market_context']}

Technical Analysis:
{technical_analysis['technical_analysis']}
"""
        
        return report

report_generator = ReportGenerator()

# Generate report based on agent outputs
report = report_generator.generate_report(
    sec_analysis_result,
    market_context_result,
    technical_analysis_result
)

print(report)

In [ ]:
def generate_visualizations(data: pd.DataFrame):
    plt.figure(figsize=(10, 6))
    
    # Plot closing prices
    plt.plot(data.index, data['Close'], label="Closing Price")
    
    plt.title("Stock Price Over Time")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    
    plt.grid(True)
    plt.show()

generate_visualizations(yahoo_finance_data)

In [ ]:
def process_financial_query(query: str):
    # Step 1: Process the query to extract key components
    processed_query = query_processor.process_query(query)

    # Step 2: Fetch data from APIs based on extracted companies and metrics
    company_tickers = [company["ticker"] for company in processed_query["companies"]]
    
    api_results_alpha_vantage = [
        api_connector.fetch_alpha_vantage_data(ticker) for ticker in company_tickers
    ]
    
    api_results_yahoo_finance = [
        api_connector.fetch_yahoo_finance_data(ticker) for ticker in company_tickers
    ]

    # Step 3: Perform analysis using specialized agents
    sec_results = [sec_agent.analyze_sec_filings(data) for data in api_results_alpha_vantage]
    
    market_results = [market_agent.gather_market_context(data) for data in api_results_yahoo_finance]
    
    technical_results = [technical_agent.perform_technical_analysis(data) for data in api_results_yahoo_finance]

    # Step 4: Generate a comprehensive report based on agent outputs
    reports = [
        report_generator.generate_report(sec_result, market_result, technical_result)
        for sec_result, market_result, technical_result in zip(sec_results, market_results, technical_results)
    ]

    # Step 5: Visualize results (example visualization for the first company's data)
    generate_visualizations(api_results_yahoo_finance[0])

    return reports

# Example usage of the workflow with a query
query_example = "Summarize TSLA's financials over the last 5 years"
final_reports = process_financial_query(query_example)

for report in final_reports:
    print(report)